In [ ]:
import os

os.environ['KERAS_BACKEND'] = 'theano'
os.environ['THEANO_FLAGS']='mode=FAST_RUN,device=cuda0,floatX=float32,optimizer=None'

import keras.models as models
from keras.models import Sequential, Model
from keras.optimizers import SGD, Adam, RMSprop
from keras.callbacks import ModelCheckpoint

import numpy as np
import glob

import matplotlib.pyplot as plt
plt.ion()
%matplotlib inline


In [ ]:
imgfiles = sorted(glob.glob('data/*-images.npz'))
steerfiles = sorted(glob.glob('data/*-steer.npz'))
# TODO: Handle more than one of each file (concat)
imgs = np.load(imgfiles[0])['img_arr']
targets = np.load(steerfiles[0])['steer_arr']

print(f'Have {imgs.shape[0]} training images')

In [ ]:
idx = 10
imgplot = plt.imshow(imgs[idx,:].astype(np.float32))
print(f'steer: {targets[idx]}')

In [ ]:
# Shuffle images and steering targets
idx = np.arange(0,imgs.shape[0])
idx = np.random.permutation(idx)
imgs = imgs[idx,:,:,:]
targets = targets[idx]

In [ ]:
# load the model:
model = Sequential()
with open('autopilot_basic_model.json') as model_file:
    model = models.model_from_json(model_file.read())


In [ ]:
# checkpoint
filepath="weights/weights.best.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

In [ ]:
adam = Adam(lr=0.0001)
model.compile(loss='mse',
              optimizer=adam,
              metrics=['mse','accuracy'])

epochs = 25
batch_size = 64

model.fit(imgs, targets, callbacks=callbacks_list,
	batch_size=batch_size, epochs=epochs, verbose=1,
	validation_split=0.1, shuffle=True)

In [ ]:
model.save_weights('weights/model_basic_weight.hdf5')